In [1]:
import pandas as pd
from tqdm import tqdm
from database.market import Market
from backtester.backtester import Backtester as b
from transformer.column_transformer import ColumnTransformer
from transformer.date_transformer import DateTransformer
from strategy.weekly_price import WeeklyPrice
from strategy.quarterly_financial_categorical import QuarterlyFinancialCategorical
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta, timezone
import math
import numpy as np

In [2]:
market = Market()
wp = WeeklyPrice()
qfc = QuarterlyFinancialCategorical()

In [3]:
weekly = wp.retrieve_data("sim_categorical").drop("_id",axis=1)
quarterly = qfc.retrieve_data("sim").drop("_id",axis=1)

In [4]:
market.connect()
prices = market.retrieve_data("prices")
market.close()

In [5]:
sim = weekly.merge(quarterly,on=["year","quarter","ticker"],how="left")

In [6]:
sim = prices.drop("_id",axis=1).merge(sim.dropna(),on = ["year","quarter","week","ticker"], how="left")

In [7]:
sim["weekly_predicted_delta"] = (sim["weekly_price_categorical_regression_prediction"] - sim["adjclose"]) / sim["adjclose"]
sim["quarterly_predicted_delta"] = (sim["quarterly_financial_categorical_regression_prediction"] - sim["adjclose"]) / sim["adjclose"]

In [8]:
sim = sim.dropna()
sim = sim.groupby(["date","ticker"]).mean().reset_index()

In [9]:
sim = sim.dropna()

In [10]:
start = datetime(2013,1,1)
end = datetime(2020,12,23)
extreme = [True]
ceiling_settings = [True]
fixed = [True,False]
classification = [True,False]
weekly_value_settings = [True,False]
quarterly_delta_range = range(10,40,10)
weekly_delta_range = range(5,15,5)
trade_signal_score_settings = range(60,90,10)
initial = 100

In [16]:
epoch = 0
seats = 5
wp.db.connect()
# wp.db.drop_table("grouped_epochs")
# iterations = len(extreme) * len(ceiling_settings) * len(fixed) * len(classification) \
#                 * len(weekly_value_settings) * len(quarterly_delta_range) * len(weekly_delta_range) \
#                  * len(trade_signal_score_settings)
# print(iterations)
# for i in range(iterations):
#     wp.db.drop_table("grouped_trades_{}".format(i))
for e in tqdm(extreme):
    for ceiling in tqdm(ceiling_settings):
        for f in tqdm(fixed):
            for c in classification:
                for weekly_value in weekly_value_settings:
                    for quarterly_delta in quarterly_delta_range:
                        for weekly_delta in weekly_delta_range:
                            for score in trade_signal_score_settings:
                                s = float(score/100)
                                qd = float(quarterly_delta/100)
                                wd = float(weekly_delta/100)
                                epoch_dict = {"epoch":epoch
                                              ,"extreme":e
                                              ,"ceiling":ceiling
                                              ,"fixed":f
                                              ,"classification":c
                                              ,"weekly_value":weekly_value
                                              ,"quarterly_delta":qd
                                              ,"weekly_delta":wd
                                              ,"score":s
                                             }
                                ts = b.backtest_weekly_adaptive(start,end,sim.copy(),seats,e,ceiling,f,c,weekly_value,qd,wd,s)
                                if len(ts) > 0:
                                    wp.db.store_data("adaptive_epochs",pd.DataFrame([epoch_dict]))
                                    wp.db.store_data("adaptive_trades_{}".format(epoch),pd.DataFrame(ts))
                                epoch += 1
wp.db.close()

  0%|                                                                                                                                                                                                                                             | 0/1 [00:00<?, ?it/s]

  0%|                                                                                                                                                                                                                                             | 0/2 [00:00<?, ?it/s]

2019-12-30 00:00:00 single positional indexer is out-of-bounds
2019-12-30 00:00:00 single positional indexer is out-of-bounds
2019-12-30 00:00:00 single positional indexer is out-of-bounds
2019-12-30 00:00:00 single positional indexer is out-of-bounds
2019-12-30 00:00:00 single positional indexer is out-of-bounds
2017-03-30 00:00:00 single positional indexer is out-of-bounds
2017-03-30 00:00:00 single positional indexer is out-of-bounds
2017-03-30 00:00:00 single positional indexer is out-of-bounds
2019-12-30 00:00:00 single positional indexer is out-of-bounds
2019-12-30 00:00:00 single positional indexer is out-of-bounds
2019-12-30 00:00:00 single positional indexer is out-of-bounds
2019-12-30 00:00:00 single positional indexer is out-of-bounds
2019-12-30 00:00:00 single positional indexer is out-of-bounds
2017-03-30 00:00:00 single positional indexer is out-of-bounds
2017-03-30 00:00:00 single positional indexer is out-of-bounds
2017-03-30 00:00:00 single positional indexer is out-of



 50%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                                 | 1/2 [20:51<20:51, 1251.77s/it]

2020-03-31 00:00:00 single positional indexer is out-of-bounds
2020-03-31 00:00:00 single positional indexer is out-of-bounds
2020-03-31 00:00:00 single positional indexer is out-of-bounds
2020-03-31 00:00:00 single positional indexer is out-of-bounds
2020-03-31 00:00:00 single positional indexer is out-of-bounds
2020-03-31 00:00:00 single positional indexer is out-of-bounds
2019-03-28 00:00:00 single positional indexer is out-of-bounds
2019-03-28 00:00:00 single positional indexer is out-of-bounds
2020-04-02 00:00:00 no more stock vets
2019-03-28 00:00:00 single positional indexer is out-of-bounds
2019-03-28 00:00:00 single positional indexer is out-of-bounds
2019-03-28 00:00:00 single positional indexer is out-of-bounds
2019-03-28 00:00:00 single positional indexer is out-of-bounds
2019-03-28 00:00:00 single positional indexer is out-of-bounds
2020-04-02 00:00:00 no more stock vets
2019-03-28 00:00:00 single positional indexer is out-of-bounds
2019-03-28 00:00:00 single positional in



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [36:20<00:00, 1090.19s/it]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [36:20<00:00, 2180.39s/it]
